In [ ]:
pip install pandas numpy folium geopy haversine gmplot shapely

In [ ]:
import numpy as np
import pandas as pd
from gmplot import gmplot
from google.colab import drive
import folium
import glob
import folium
import pandas as pd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import folium
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# speed, density 데이터 불러오기
sd_20210111_file_path = '/content/drive/MyDrive/20210111'
sd_20210113_file_path = '/content/drive/MyDrive/20210113'
sd_20210114_file_path = '/content/drive/MyDrive/20210114'

# MyDrive의 data 폴더에서 모든 CSV 파일 경로 가져오기
sd_20210111_csv_files = glob.glob('/content/drive/MyDrive/20210111/*.csv')
sd_20210113_csv_files = glob.glob('/content/drive/MyDrive/20210113/*.csv')
sd_20210114_csv_files = glob.glob('/content/drive/MyDrive/20210114/*.csv')

In [ ]:
# CSV 파일을 처리하는 함수
def load_and_merge_files(csv_files):
    speed_file = None
    density_file = None

    # 속도 파일과 밀도 파일 구분하여 로드
    for file in csv_files:
        if 'VHCLE_SPD' in file:  # 속도 파일
            speed_file = pd.read_csv(file)
            print(f"Speed file columns: {list(speed_file.columns)}")  # 속도 파일의 컬럼 출력
        elif 'VHCLE_DENS' in file:  # 밀도 파일
            density_file = pd.read_csv(file)
            print(f"Density file columns: {list(density_file.columns)}")  # 밀도 파일의 컬럼 출력

    # 속도 파일과 밀도 파일이 모두 존재하는지 확인
    if speed_file is not None and density_file is not None:
        # 두 파일을 공통된 키로 결합
        merged_data = pd.merge(
            density_file,
            speed_file,
            on=['STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN'],
            how='inner'
        )

        # 컬럼 존재 여부 확인 후 선택
        required_columns = [
            'STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN',
            'VHCLE_KPH', 'VHCLE_DENS',
            'RRGMNT_WGS84_LAT', 'RRGMNT_WGS84_LONG',
            'RRGMNT_UTMK_Y_CRDNT', 'RRGMNT_UTMK_X_CRDNT',
            'RRGMNT_1ST_TLLBADD', 'RRGMNT_2ND_TLLBADD', 'RRGMNT_3RD_TLLBADD'
        ]

        # 실제 존재하는 컬럼들로 필터링
        available_columns = [col for col in required_columns if col in merged_data.columns]

        # 필요한 컬럼만 선택
        merged_data = merged_data[available_columns]
        return merged_data
    else:
        print("속도 파일이나 밀도 파일 중 하나가 없습니다.")
        return None

# 각 날짜별로 파일을 결합
merged_20210111 = load_and_merge_files(sd_20210111_csv_files)
merged_20210113 = load_and_merge_files(sd_20210113_csv_files)
merged_20210114 = load_and_merge_files(sd_20210114_csv_files)

# 각 결합된 데이터 확인
if merged_20210111 is not None:
    print("2021-01-11 결합 데이터:")
    print(merged_20210111.head())

if merged_20210113 is not None:
    print("2021-01-13 결합 데이터:")
    print(merged_20210113.head())

if merged_20210114 is not None:
    print("2021-01-14 결합 데이터:")
    print(merged_20210114.head())


Speed file columns: ['STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN', 'VHCLE_KPH', 'RRGMNT_WGS84_LAT', 'RRGMNT_WGS84_LONG', 'RRGMNT_UTMK_Y_CRDNT', 'RRGMNT_UTMK_X_CRDNT', 'RRGMNT_1ST_TLLBADD', 'RRGMNT_2ND_TLLBADD', 'RRGMNT_3RD_TLLBADD']
Density file columns: ['STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN', 'VHCLE_DENS', 'RRGMNT_WGS84_LAT', 'RRGMNT_WGS84_LONG', 'RRGMNT_UTMK_Y_CRDNT', 'RRGMNT_UTMK_X_CRDNT', 'RRGMNT_1ST_TLLBADD', 'RRGMNT_2ND_TLLBADD', 'RRGMNT_3RD_TLLBADD']
Speed file columns: ['STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN', 'VHCLE_KPH', 'RRGMNT_WGS84_LAT', 'RRGMNT_WGS84_LONG', 'RRGMNT_UTMK_Y_CRDNT', 'RRGMNT_UTMK_X_CRDNT', 'RRGMNT_1ST_TLLBADD', 'RRGMNT_2ND_TLLBADD', 'RRGMNT_3RD_TLLBADD']
Density file columns: ['STNDR_DAYHMINSEC', 'CCTV_GPS_LAT', 'CCTV_GPS_LONG', 'CCTV_LOCA_EXPLN', 'VHCLE_DENS', 'RRGMNT_WGS84_LAT', 'RRGMNT_WGS84_LONG', 'RRGMNT_UTMK_Y_CRDNT', 'RRGMNT_UTMK_X_CRDNT', 'RRGMNT_1ST_TLLBADD', 'R

In [ ]:
merged_20210111

,STNDR_DAYHMINSEC,CCTV_GPS_LAT,CCTV_GPS_LONG,CCTV_LOCA_EXPLN,VHCLE_KPH,VHCLE_DENS
0,20210111000000,35.187818,128.086681,진주교 교차로,0,0
1,20210111000000,35.187818,128.086681,진주교 교차로,0,0
2,20210111000000,35.187818,128.086681,진주교 교차로,0,0
3,20210111000000,35.187818,128.086681,진주교 교차로,0,0
4,20210111000005,35.187818,128.086681,진주교 교차로,0,0
...,...,...,...,...,...,...
3260731,20210111235531,35.179997,128.117429,소방서 방향,0,0
3260732,20210111235925,35.179997,128.117429,소방서 방향,0,0
3260733,20210111235925,35.179997,128.117429,소방서 방향,0,0
3260734,20210111235925,35.179997,128.117429,소방서 방향,0,0


In [ ]:
# 축제 차량 통제 구간 안에 포함된 CCTV 좌표

latitudes_to_remove = [35.191305, 35.191226, 35.191261, 35.19134, 35.187818, 35.186102, 35.193139]

# CCTV_GPS_LAT이 위도 리스트에 포함되지 않은 데이터만 필터링
merged_20210111_filtered = merged_20210111[~merged_20210111['CCTV_GPS_LAT'].isin(latitudes_to_remove)]
merged_20210113_filtered = merged_20210113[~merged_20210113['CCTV_GPS_LAT'].isin(latitudes_to_remove)]
merged_20210114_filtered = merged_20210114[~merged_20210114['CCTV_GPS_LAT'].isin(latitudes_to_remove)]

In [ ]:
# 1. 세부 데이터프레임을 나누고 속도와 밀도 데이터를 추출하는 함수 정의
def process_grouped_data(grouped_df):
    sub_dataframes = {}
    for (lat, lng), group in grouped_df:
        df_name = f"df_{lat}_{lng}"
        sub_dataframes[df_name] = group
    return sub_dataframes

# 2. 각 날짜의 데이터프레임을 CCTV_GPS_LAT와 CCTV_GPS_LONG 값으로 나누기
grouped_data = {
    '20210111': merged_20210111_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
    '20210113': merged_20210113_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
    '20210114': merged_20210114_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
}

# 3. 세부 데이터프레임을 생성하고 속도와 밀도 데이터를 추출하는 코드
data_per_day = {}
for date, grouped_df in grouped_data.items():
    sub_dataframes = process_grouped_data(grouped_df)
    data_per_day[date] = {name: (df['VHCLE_KPH'].tolist(), df['VHCLE_DENS'].tolist()) for name, df in sub_dataframes.items()}

# 4. 추출된 데이터 확인 (속도, 밀도)
for date, data in data_per_day.items():
    for name, (speed_data, density_data) in data.items():
        print(f"Location {name}: Speed data (first 5) {speed_data[:5]}, Density data (first 5) {density_data[:5]}")


Location df_35.179053_128.091339: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179166_128.101039: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179192_128.10132: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179236_128.100953: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179277_128.10124: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179454_128.10696: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179489_128.107054: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179534_128.106918: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179564_128.107003: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Loca

In [ ]:
# 1. 데이터프레임을 CCTV_GPS_LAT와 CCTV_GPS_LONG 값으로 나누기
grouped_20210111 = merged_20210111_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG'])
grouped_20210113 = merged_20210113_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG'])
grouped_20210114 = merged_20210114_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG'])

# 세부 데이터프레임 저장을 위한 딕셔너리 생성
sub_dataframes_20210111 = {}
sub_dataframes_20210113 = {}
sub_dataframes_20210114 = {}

# 각 그룹별로 세부 데이터프레임 생성
for (lat, lng), group in grouped_20210111:
    df_name = f"df_{lat}_{lng}"  # 세부 데이터프레임 이름을 CCTV_GPS_LAT와 CCTV_GPS_LONG 값으로 설정
    sub_dataframes_20210111[df_name] = group  # 딕셔너리에 저장

for (lat, lng), group in grouped_20210113:
    df_name = f"df_{lat}_{lng}"
    sub_dataframes_20210113[df_name] = group

for (lat, lng), group in grouped_20210114:
    df_name = f"df_{lat}_{lng}"
    sub_dataframes_20210114[df_name] = group

# 2. 속도(VHCLE_KPH)와 밀도(VHCLE_DENS) 컬럼을 추출하여 도로 한계 수용량 추정 모델에 활용
for name, df in sub_dataframes_20210111.items():
    speed_data_20210111 = df['VHCLE_KPH'].tolist()
    density_data_20210111 = df['VHCLE_DENS'].tolist()

for name, df in sub_dataframes_20210113.items():
    speed_data_20210113 = df['VHCLE_KPH'].tolist()
    density_data_20210113 = df['VHCLE_DENS'].tolist()

for name, df in sub_dataframes_20210114.items():
    speed_data_20210114 = df['VHCLE_KPH'].tolist()
    density_data_20210114 = df['VHCLE_DENS'].tolist()

In [ ]:
import pandas as pd

# 1. 세부 데이터프레임을 나누고 속도와 밀도 데이터를 추출하는 함수 정의
def process_grouped_data(grouped_df):
    sub_dataframes = {}
    for (lat, lng), group in grouped_df:
        df_name = f"df_{lat}_{lng}"
        sub_dataframes[df_name] = group
    return sub_dataframes

# 2. 각 날짜의 데이터프레임을 CCTV_GPS_LAT와 CCTV_GPS_LONG 값으로 나누기
grouped_data = {
    '20210111': merged_20210111_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
    '20210113': merged_20210113_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
    '20210114': merged_20210114_filtered.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG']),
}

# 3. 세부 데이터프레임을 생성하고 속도와 밀도 데이터를 추출하는 코드
data_per_day = {}
for date, grouped_df in grouped_data.items():
    sub_dataframes = process_grouped_data(grouped_df)
    data_per_day[date] = {name: (df['VHCLE_KPH'].tolist(), df['VHCLE_DENS'].tolist()) for name, df in sub_dataframes.items()}

# 4. 추출된 데이터 확인 (속도, 밀도)
for date, data in data_per_day.items():
    print(f"Data for {date}:")
    for name, (speed_data, density_data) in data.items():
        print(f"Location {name}: Speed data (first 5) {speed_data[:5]}, Density data (first 5) {density_data[:5]}")

Data for 20210111:
Location df_35.179053_128.091339: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179166_128.101039: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179192_128.10132: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179236_128.100953: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179277_128.10124: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179454_128.10696: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179489_128.107054: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179534_128.106918: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [0, 0, 0, 0, 0]
Location df_35.179564_128.107003: Speed data (first 5) [0, 0, 0, 0, 0], Density data (first 5) [

In [ ]:
# 각 세부 데이터프레임의 속도와 밀도 데이터를 도로 한계 수용량 추정 모델에 넣는 코드
for name, df in sub_dataframes.items():
    # 1. 속도와 밀도 데이터 추출
    vhcle_speed_list = df['VHCLE_KPH'].tolist()
    vhcle_dens_list = df['VHCLE_DENS'].tolist()

    # 속도 데이터가 0이면 이동 제한 속도인 50으로 처리
    vhcle_speed_list = [50 if v == 0 else v for v in vhcle_speed_list]

    actual_speeds = np.array(vhcle_speed_list)  # 차량 실제 속도 배열
    densities = np.array(vhcle_dens_list)  # 교통 밀집도 배열

    # 초기 추정치 설정 (임계 밀도, 자유 흐름 속도, 지수 매개변수)
    initial_params = [20, 50, 0.3]

    # 여러 번 실행한 도로 수용량을 저장할 리스트
    ith_capacities = []

    # 유전자 알고리즘을 100번 실행하여 결과 수집
    for _ in range(100):
        calibrated_params, _ = genetic_algorithm_calibration(actual_speeds, densities, initial_params)
        capacity = estimate_road_capacity(calibrated_params, densities.mean(), num_lanes=2)
        ith_capacities.append(capacity)

    # 평균 도로 수용량 계산
    estimated_capacity = np.mean(ith_capacities)

    # 결과 출력
    print(f"Estimated Road Capacity for {name}: {estimated_capacity:.2f} vehicles/hour")

Estimated Road Capacity for df_35.179053_128.091339: 92.98 vehicles/hour
Estimated Road Capacity for df_35.179166_128.101039: 96.32 vehicles/hour
Estimated Road Capacity for df_35.179192_128.10132: 96.77 vehicles/hour
Estimated Road Capacity for df_35.179236_128.100953: 97.06 vehicles/hour
Estimated Road Capacity for df_35.179277_128.10124: 96.43 vehicles/hour
Estimated Road Capacity for df_35.179454_128.10696: 96.72 vehicles/hour
Estimated Road Capacity for df_35.179489_128.107054: 96.99 vehicles/hour
Estimated Road Capacity for df_35.179534_128.106918: 96.93 vehicles/hour
Estimated Road Capacity for df_35.179564_128.107003: 96.85 vehicles/hour
Estimated Road Capacity for df_35.179685_128.112206: 96.71 vehicles/hour
Estimated Road Capacity for df_35.179692_128.112208: 97.05 vehicles/hour
Estimated Road Capacity for df_35.179732_128.112298: 95.84 vehicles/hour
Estimated Road Capacity for df_35.179736_128.112308: 92.31 vehicles/hour
Estimated Road Capacity for df_35.17977_128.112144: 96

In [ ]:
# 도로 한계 수용량 추정 모델
def traffic_flow_model(density, critical_density, free_flow_speed, exponential_param):
    return free_flow_speed * np.exp(-exponential_param * (density / critical_density))

def fitness_function(actual_speed, predicted_speed):
    return np.mean((actual_speed - predicted_speed) ** 2)

def genetic_algorithm_calibration(actual_speeds, densities, initial_params, iterations=100):
    population_size = 30
    best_fitness = float('inf')
    best_params = None

    for iteration in range(iterations):
        population = np.random.rand(population_size, 3) * initial_params
        fitness_scores = []

        for params in population:
            predicted_speeds = traffic_flow_model(densities, *params)
            fitness = fitness_function(actual_speeds, predicted_speeds)
            fitness_scores.append(fitness)

            if fitness < best_fitness:
                best_fitness = fitness
                best_params = params

        best_population = np.array(population)[np.argsort(fitness_scores)][:10]
        worst_population = np.random.rand(10, 3) * initial_params
        population = np.vstack((best_population, worst_population))

        if np.abs(np.min(fitness_scores) - best_fitness) < 0.001:
            break

    return best_params, best_fitness

def estimate_road_capacity(calibrated_params, lane_density, num_lanes=5):
    critical_density, free_flow_speed, exponential_param = calibrated_params
    lane_capacity = traffic_flow_model(lane_density, critical_density, free_flow_speed, exponential_param)
    total_capacity = lane_capacity * num_lanes
    return total_capacity


In [ ]:
# 1. 데이터프레임을 CCTV_GPS_LAT와 CCTV_GPS_LONG 값으로 나누는 함수 정의
def split_by_location(df):
    grouped = df.groupby(['CCTV_GPS_LAT', 'CCTV_GPS_LONG'])
    sub_dataframes = {}
    for (lat, lng), group in grouped:
        df_name = f"df_{lat}_{lng}"
        sub_dataframes[df_name] = group
    return sub_dataframes

# 2. 도로 한계 수용량 추정 관련 함수들
def traffic_flow_model(density, critical_density, free_flow_speed, exponential_param):
    return free_flow_speed * np.exp(-exponential_param * (density / critical_density))

def fitness_function(actual_speed, predicted_speed):
    return np.mean((actual_speed - predicted_speed) ** 2)

def genetic_algorithm_calibration(actual_speeds, densities, initial_params, iterations=100):
    population_size = 30
    best_fitness = float('inf')
    best_params = None

    for iteration in range(iterations):
        population = np.random.rand(population_size, 3) * initial_params
        fitness_scores = []

        for params in population:
            predicted_speeds = traffic_flow_model(densities, *params)
            fitness = fitness_function(actual_speeds, predicted_speeds)
            fitness_scores.append(fitness)

            if fitness < best_fitness:
                best_fitness = fitness
                best_params = params

        best_population = np.array(population)[np.argsort(fitness_scores)][:10]
        worst_population = np.random.rand(10, 3) * initial_params
        population = np.vstack((best_population, worst_population))

        if np.abs(np.min(fitness_scores) - best_fitness) < 0.001:
            break

    return best_params, best_fitness

def estimate_road_capacity(calibrated_params, lane_density, num_lanes=5):
    critical_density, free_flow_speed, exponential_param = calibrated_params
    lane_capacity = traffic_flow_model(lane_density, critical_density, free_flow_speed, exponential_param)
    total_capacity = lane_capacity * num_lanes
    return total_capacity

# 3. 각 날짜별 데이터를 처리한 후 결과를 결합하는 함수
def process_all_days(dataframes_dict):
    capacities_by_location = {}

    for date, df in dataframes_dict.items():
        print(f"Processing data for {date}")
        sub_dataframes = split_by_location(df)

        for name, df in sub_dataframes.items():
            vhcle_speed_list = df['VHCLE_KPH'].tolist()
            vhcle_dens_list = df['VHCLE_DENS'].tolist()

            # 속도 데이터가 0이면 이동 제한 속도인 50으로 처리
            vhcle_speed_list = [50 if v == 0 else v for v in vhcle_speed_list]

            actual_speeds = np.array(vhcle_speed_list)  # 차량 실제 속도 배열
            densities = np.array(vhcle_dens_list)  # 교통 밀집도 배열

            # 초기 추정치 설정 (임계 밀도, 자유 흐름 속도, 지수 매개변수)
            initial_params = [20, 50, 0.3]

            # 여러 번 실행한 도로 수용량을 저장할 리스트
            ith_capacities = []

            # 유전자 알고리즘을 100번 실행하여 결과 수집
            for _ in range(100):
                calibrated_params, _ = genetic_algorithm_calibration(actual_speeds, densities, initial_params)
                capacity = estimate_road_capacity(calibrated_params, densities.mean(), num_lanes=2)
                ith_capacities.append(capacity)

            # 평균 도로 수용량 계산
            estimated_capacity = np.mean(ith_capacities)

            # 각 위치별 도로 수용량 저장
            if name not in capacities_by_location:
                capacities_by_location[name] = []
            capacities_by_location[name].append(estimated_capacity)

    return capacities_by_location

# 4. 최종 결과 출력 (각 좌표별 평균 도로 수용량)
def print_average_capacities(capacities_by_location):
    print("Average Road Capacity by Location (vehicles/hour):")
    for location, capacities in capacities_by_location.items():
        average_capacity = np.mean(capacities)
        print(f"Location {location}: {average_capacity:.2f} vehicles/hour")


# 데이터프레임 로드 예시 (merged_20210111, merged_20210113, merged_20210114가 주어졌다고 가정)
dataframes_dict = {
    '20210111': merged_20210111_filtered,
    '20210113': merged_20210113_filtered,
    '20210114': merged_20210114_filtered
}

# 5. 각 날짜에 대한 도로 수용량 추정 실행
capacities_by_location = process_all_days(dataframes_dict)

# 6. 최종 결과 출력
print_average_capacities(capacities_by_location)


Processing data for 20210111
Processing data for 20210113
Processing data for 20210114
Average Road Capacity by Location (vehicles/hour):
Location df_35.179053_128.091339: 92.91 vehicles/hour
Location df_35.179166_128.101039: 96.94 vehicles/hour
Location df_35.179192_128.10132: 96.67 vehicles/hour
Location df_35.179236_128.100953: 96.71 vehicles/hour
Location df_35.179277_128.10124: 96.73 vehicles/hour
Location df_35.179454_128.10696: 96.82 vehicles/hour
Location df_35.179489_128.107054: 96.99 vehicles/hour
Location df_35.179534_128.106918: 96.90 vehicles/hour
Location df_35.179564_128.107003: 96.89 vehicles/hour
Location df_35.179685_128.112206: 96.78 vehicles/hour
Location df_35.179692_128.112208: 96.70 vehicles/hour
Location df_35.179732_128.112298: 95.69 vehicles/hour
Location df_35.179736_128.112308: 91.81 vehicles/hour
Location df_35.17977_128.112144: 96.76 vehicles/hour
Location df_35.179776_128.112134: 96.71 vehicles/hour
Location df_35.179804_128.112244: 96.64 vehicles/hour
Lo

In [ ]:
east_parking_lot = pd.DataFrame({
    'name': ['진주종합운동장', '물초원공원', '윙스타워', '연암공대', '진주역'],
    'latitude': [35.18251, 35.18186, 35.16580, 35.16380, 35.15208],
    'longitude': [128.1382, 128.1457, 128.1317, 128.1051, 128.1188]
})

# CCTV 위치 데이터
cctv_location = pd.DataFrame({
    'name': ['1번', '2번', '3번', '4번', '5번', '6번'],
    'latitude': [35.178, 35.187972, 35.18103, 35.179194, 35.17117, 35.180028],
    'longitude': [128.09133, 128.086694, 128.1381, 128.101333, 128.0949, 128.117528]
})

school_locations = pd.DataFrame({
    'name': ['금성초등학교', '동명중고등학교', '진주중앙고등학교', '진주제일여자고등학교', '도동초등학교', '진주중앙중학교', '초전초등학교'],
    'latitude': [35.19789, 35.20266, 35.19351, 35.19065, 35.18955, 35.19044, 35.20754],
    'longitude': [128.1150, 128.1192, 128.1186, 128.1149, 128.1175, 128.1254, 128.1145]
})

# 축제지 중심 좌표와 반경
festival_center_point = (35.188595, 128.081380)
festival_radius_km = 1

# 지도 초기화 (중심 좌표는 축제지 중심으로 설정)
m = folium.Map(location=festival_center_point, zoom_start=13)

# 축제지 중심 마커 추가
folium.Marker(
    location=festival_center_point,
    popup="축제지 중심",
    icon=folium.Icon(color='green', icon='star')
).add_to(m)

# 1번 작업: east_parking_lot의 각 좌표에 반경 1km 원과 주차장 이름 표시
for idx, row in east_parking_lot.iterrows():
    parking_location = (row['latitude'], row['longitude'])

    # 반경 1km 원과 마커 추가
    folium.Circle(
        location=parking_location,
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
        popup=row['name']  # 주차장 이름 표시
    ).add_to(m)

    # 주차장 위치에 마커 추가
    folium.Marker(
        location=parking_location,
        popup=row['name'],  # 주차장 이름 말풍선
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# 2번 - 7번과 1번 - 3번을 파란색으로 연결하며 팝업 추가
connections_with_popups_blue = [
    ((1, 4), "Count: 84 / 198"),
    ((0, 2), "Count: 266 / 907")
]

for (start, end), popup_text in connections_with_popups_blue:
    folium.PolyLine(
        locations=[
            [cctv_location.iloc[start]['latitude'], cctv_location.iloc[start]['longitude']],
            [cctv_location.iloc[end]['latitude'], cctv_location.iloc[end]['longitude']]
        ],
        color="blue",
        weight=10,
        popup=popup_text
    ).add_to(m)

# 3번 작업: 각 CCTV 위치에 마커 추가 (1번~10번)
for idx, row in cctv_location.iterrows():
    cctv_location_point = (row['latitude'], row['longitude'])

    # CCTV 위치에 마커 추가
    folium.Marker(
        location=cctv_location_point,
        popup=f"CCTV {row['name']} 위치",  # CCTV 번호 표시
        icon=folium.Icon(color='green', icon='camera')
    ).add_to(m)

# 학교 마커 표시 코드

# 학교 위치에 반경 1km 원과 마커 추가
for idx, row in school_locations.iterrows():
    school_location_point = (row['latitude'], row['longitude'])

    # 학교 위치에 1km 반경의 주황색 원 추가
    folium.Circle(
        location=school_location_point,
        radius=500,  # 반경 500m
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.3,
        popup=row['name']  # 학교 이름 표시
    ).add_to(m)

    # 학교 위치에 마커 추가
    folium.Marker(
        location=school_location_point,
        popup=row['name'],  # 학교 이름 표시
        icon=folium.Icon(color='red', icon='graduation-cap')
    ).add_to(m)

# 지도 저장 및 표시
m.save('east_parking_cctv_map_with_festival_center.html')
m
